In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import gdown
%cd gdrive/My Drive/NAFNet

!git clone https://github.com/megvii-research/NAFNet
%cd NAFNet
!pip install -r requirements.txt
!pip install --upgrade --no-cache-dir gdown
!python3 setup.py develop --no_cuda_ext

gdown.download('https://drive.google.com/uc?id=14Fht1QQJ2gMlk4N1ERCRuElg8JfjrWWR', "./experiments/pretrained_models/", quiet=False)

[Errno 2] No such file or directory: 'gdrive/My Drive/NAFNet'
/content
Cloning into 'NAFNet'...
remote: Enumerating objects: 389, done.
remote: Counting objects: 100% (389/389), done.
remote: Compressing objects: 100% (248/248), done.
remote: Total 389 (delta 195), reused 288 (delta 126), pack-reused 0
Receiving objects: 100% (389/389), 8.88 MiB | 4.10 MiB/s, done.
Resolving deltas: 100% (195/195), done.
/content/NAFNet
     |████████████████████████████████| 5.8 MB 14.2 MB/s 
     |████████████████████████████████| 190 kB 63.4 MB/s 
running develop
running egg_info
creating basicsr.egg-info
writing basicsr.egg-info/PKG-INFO
writing dependency_links to basicsr.egg-info/dependency_links.txt
writing top-level names to basicsr.egg-info/top_level.txt
writing manifest file 'basicsr.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'basicsr.egg-info/SOURCES.txt'
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:387: UserWarning: Attempted to use ninj

Downloading...
From: https://drive.google.com/uc?id=14Fht1QQJ2gMlk4N1ERCRuElg8JfjrWWR
To: /content/NAFNet/experiments/pretrained_models/NAFNet-SIDD-width64.pth
100%|██████████| 464M/464M [00:07<00:00, 64.2MB/s]


'./experiments/pretrained_models/NAFNet-SIDD-width64.pth'

In [ ]:
import torch

from basicsr.models import create_model
from basicsr.utils import img2tensor as _img2tensor, tensor2img, imwrite
from basicsr.utils.options import parse
import numpy as np
import cv2
import matplotlib.pyplot as plt

def imread(img_path):
  img = cv2.imread(img_path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img
def img2tensor(img, bgr2rgb=False, float32=True):
    img = img.astype(np.float32) / 255.
    return _img2tensor(img, bgr2rgb=bgr2rgb, float32=float32)

def display(img1, img2):
  fig = plt.figure(figsize=(25, 10))
  ax1 = fig.add_subplot(1, 2, 1) 
  plt.title('Input image', fontsize=16)
  ax1.axis('off')
  ax2 = fig.add_subplot(1, 2, 2)
  plt.title('NAFNet output', fontsize=16)
  ax2.axis('off')
  ax1.imshow(img1)
  ax2.imshow(img2)

def single_image_inference(model, img, save_path):
      model.feed_data(data={'lq': img.unsqueeze(dim=0)})

      if model.opt['val'].get('grids', False):
          model.grids()

      model.test()

      if model.opt['val'].get('grids', False):
          model.grids_inverse()

      visuals = model.get_current_visuals()
      sr_img = tensor2img([visuals['result']])
      imwrite(sr_img, save_path)

In [ ]:
opt_path = 'options/test/SIDD/NAFNet-width64.yml'
opt = parse(opt_path, is_train=False)
opt['dist'] = False
NAFNet = create_model(opt)

 load net keys <built-in method keys of collections.OrderedDict object at 0x7f2998a54e60>


2022-04-26 03:29:16,304 INFO: Model [ImageRestorationModel] is created.


In [ ]:
import os
input_path = '/content/gdrive/MyDrive/DL_Project/image_filtering/output'
output_path = '/content/gdrive/MyDrive/DL_Project/image_filtering/other_test/NAF_NET'
# img_input = imread(input_path)
# inp = img2tensor(img_input)
# single_image_inference(NAFNet, inp, output_path)
# img_output = imread(output_path)
# display(img_input, img_output)

for seq in range(1,6):
    file = ('out_%s.png' % (seq) )
    filename = os.path.join(input_path, file)
    print(filename)
    img_input=imread(filename)
    inp = img2tensor(img_input)
    outfile=('out_%s.png' % (seq) )
    out=os.path.join(input_path, outfile)
    single_image_inference(NAFNet, inp, out)
    img_output = imread(out)
    display(img_input, img_output)

Output hidden; open in https://colab.research.google.com to view.